In [ ]:
import numpy as np 
import cv2
from matplotlib import pyplot as plt
import os

In [15]:
import numpy as np 
import cv2
net = cv2.dnn.readNetFromCaffe('./model/colorization_deploy_v2.prototxt','./model/colorization_release_v2.caffemodel')
pts = np.load('./model/pts_in_hull.npy')


class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2,313,1,1)

net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1,313],2.606,dtype='float32')]

In [ ]:
def video(frame):
    scaled = frame.astype("float32")/255.0
    lab = cv2.cvtColor(scaled,cv2.COLOR_BGR2LAB)

    resized = cv2.resize(lab,(224,224))
    L = cv2.split(resized)[0]
    L -= 50

    net.setInput(cv2.dnn.blobFromImage(L))
    ab = net.forward()[0, :, :, :].transpose((1,2,0))
    ab = cv2.resize(ab, (frame.shape[1],frame.shape[0]))

    L = cv2.split(lab)[0]
    colorized = np.concatenate((L[:,:,np.newaxis], ab), axis=2)
    colorized = cv2.cvtColor(colorized,cv2.COLOR_LAB2BGR)
    colorized = np.clip(colorized,0,1)
    colorized = (255 * colorized).astype("uint8")
    colorized = cv2.resize(colorized, (224, 224)) 
    return colorized

In [3]:
os.path.join('videos', 'video.mp4')

'videos\\video.mp4'

In [11]:
cap = cv2.VideoCapture('video.mp4')

In [24]:
cap = cv2.VideoCapture('output-resize.mp4')

In [ ]:
ret, frame = cap.read()

In [ ]:
frame.shape

In [ ]:
plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
plt.show()

In [25]:
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH )
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT )
fps =  cap.get(cv2.CAP_PROP_FPS)

In [26]:
h, w, c = frame.shape
print('width:  ', w)
print('height: ', h)
print('channel:', c)
# width:   400
# height:  225
# channel: 3

width:   224
height:  224
channel: 3


In [16]:
cap.get(cv2.CAP_PROP_FPS)

25.0

In [2]:
cap = cv2.VideoCapture('video.mp4')

for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
    ret, frame = cap.read()
    cv2.imshow("Video Player", frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [32]:
cap = cv2.VideoCapture('output-resize.mp4')
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = cap.get(cv2.CAP_PROP_FPS)
video_writer = cv2.VideoWriter(os.path.join('output.mp4'), cv2.VideoWriter_fourcc('P','I','M','1'), 15, (224, 224), isColor=False) 
                                                                                        
for frame_idx in range(300):
    
    ret, frame = cap.read()

    colorized = video(frame)

    video_writer.write(colorized)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
video_writer.release()

In [26]:
cap = cv2.VideoCapture('video.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output-resize.mp4',fourcc, fps, (224,224))

for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
    ret, frame = cap.read()
    if ret == True:
        b = cv2.resize(frame,(224,224),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
        out.write(b)
    else:
        break
    
cap.release()
out.release()
cv2.destroyAllWindows()